In [ ]:
import gspread
import json
import numpy as np
from scipy import stats
import pandas  as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
from oauth2client.service_account import ServiceAccountCredentials
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name('I:/gcp_auth/spreadsheet-service-302407.json', scope)
gc = gspread.authorize(credentials)
SPREADSHEET_KEY = '*'
worksheets = gc.open_by_key(SPREADSHEET_KEY)
data_sheet = worksheets.worksheet('shCtrl_003_shOPA1_003(論文用データ)')

In [ ]:
kawai_df = pd.read_csv("Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/data/compare_OPA1/PC_analyzed_without_angle.csv", index_col=0)

In [ ]:
mito_V_LST  = data_sheet.col_values(8)[1:]
mito_No_LST  = data_sheet.col_values(3)[1:]
cropped_No_LST =  data_sheet.col_values(2)[1:]


In [ ]:
mito_No_lst = []
cropped_No_lst = []


for mito_V in list(kawai_df.mito_V_px):
    for i in range(len(mito_V_LST)):
        if mito_V == int(mito_V_LST[i]):
            mito_No_lst.append(int(mito_No_LST[i]))
            cropped_No_lst.append(int(cropped_No_LST[i]))
        

In [ ]:
kawai_df["mito_No"] = mito_No_lst
kawai_df["cropped_No"] = cropped_No_lst

In [ ]:
# PC1 : 0 ~ 1
group1 = []
for node in kawai_df.itertuples():
    if node.PC1 > 0 and node.PC1 < 1:
        group1.append({
            "Gene" : node.Gene,
            "mito_No" : node.mito_No,
            "cropped_No" : node.cropped_No
        })
        
group1

In [ ]:
# PC1 : -2.5 ~ -1.5
# PC2 : 3 ~ 4
group2 = []
for node in kawai_df.itertuples():
    if node.PC1 > -2.5 and node.PC1 < -1.5 and node.PC2 > 3 and node.PC2 < 4:
        group2.append({
            "Gene" : node.Gene,
            "mito_No" : node.mito_No,
            "cropped_No" : node.cropped_No
        })
        
group2

In [ ]:
CROP_NO = 1


mito_imgs, _ =  load_Y_gray(f"Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_00{CROP_NO}/devided_mito/", normalize=False)
lamellar_imgs, _ = load_Y_gray(f"Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_00{CROP_NO}/merged_cristae_final/lamellar_with_postprocessing/", normalize=False)
tubular_imgs, _ = load_Y_gray(f"Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_00{CROP_NO}/merged_cristae_final/tubular_with_postprocessing/", normalize=False)

labeled_mito_imgs = cc3d.connected_components(mito_imgs[:,:,:,0].astype(int),  connectivity=6)


lst = list()
for node in group2:
    if node["Gene"] == "shCtrl" and node["cropped_No"] == CROP_NO:
        lst.append(node["mito_No"])
        
for num in lst:
    target_mito_coord = np.nonzero(labeled_mito_imgs==num)
    merged_imgs_mito = np.where(
        labeled_mito_imgs[
            np.min(target_mito_coord[0]) - 2 : np.max(target_mito_coord[0]) + 2,
            np.min(target_mito_coord[1]) - 2 : np.max(target_mito_coord[1]) + 2,
            np.min(target_mito_coord[2]) - 2 : np.max(target_mito_coord[2]) + 2
        ] == num,
        1,
        0
    )[..., np.newaxis]

    merged_imgs_lamellar = np.where(
        merged_imgs_mito == 1,
        lamellar_imgs[
            np.min(target_mito_coord[0]) - 2 : np.max(target_mito_coord[0]) + 2,
            np.min(target_mito_coord[1]) - 2 : np.max(target_mito_coord[1]) + 2,
            np.min(target_mito_coord[2]) - 2 : np.max(target_mito_coord[2]) + 2
        ],
        0
    )

    merged_imgs_tubular = np.where(
        merged_imgs_mito == 1,
        tubular_imgs[
            np.min(target_mito_coord[0]) - 2 : np.max(target_mito_coord[0]) + 2,
            np.min(target_mito_coord[1]) - 2 : np.max(target_mito_coord[1]) + 2,
            np.min(target_mito_coord[2]) - 2 : np.max(target_mito_coord[2]) + 2
        ],
        0
    )

    merged_imgs = np.concatenate(
        [merged_imgs_mito, merged_imgs_lamellar, merged_imgs_tubular],
        axis=3
    )
    
    out_dir =  f"Z:/paper_data/2021_cristae/correspose_PCA_without_angle/group1/shCtrl_cropped00{CROP_NO}_mito{str(num).zfill(3)}/imgs"
    os.makedirs(out_dir)

    for i in range(merged_imgs.shape[0]):
        cv2.imwrite(
            f"{out_dir}/{str(i).zfill(4)}.png",
            merged_imgs[i]
        )
    
    

del mito_imgs, lamellar_imgs, tubular_imgs, labeled_mito_imgs

In [ ]:
CROP_NO = 2


mito_imgs, _ =  load_Y_gray(f"Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_00{CROP_NO}/devided_mito/", normalize=False)
lamellar_imgs, _ = load_Y_gray(f"Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_00{CROP_NO}/merged_cristae_final/lamellar_with_postprocessing/", normalize=False)
tubular_imgs, _ = load_Y_gray(f"Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_00{CROP_NO}/merged_cristae_final/tubular_with_postprocessing/", normalize=False)

labeled_mito_imgs = cc3d.connected_components(mito_imgs[:,:,:,0].astype(int),  connectivity=6)


lst = list()
for node in group2:
    if node["Gene"] == "shCtrl" and node["cropped_No"] == CROP_NO:
        lst.append(node["mito_No"])
        
for num in lst:
    target_mito_coord = np.nonzero(labeled_mito_imgs==num)
    merged_imgs_mito = np.where(
        labeled_mito_imgs[
            np.min(target_mito_coord[0]) - 2 : np.max(target_mito_coord[0]) + 2,
            np.min(target_mito_coord[1]) - 2 : np.max(target_mito_coord[1]) + 2,
            np.min(target_mito_coord[2]) - 2 : np.max(target_mito_coord[2]) + 2
        ] == num,
        1,
        0
    )[..., np.newaxis]

    merged_imgs_lamellar = np.where(
        merged_imgs_mito == 1,
        lamellar_imgs[
            np.min(target_mito_coord[0]) - 2 : np.max(target_mito_coord[0]) + 2,
            np.min(target_mito_coord[1]) - 2 : np.max(target_mito_coord[1]) + 2,
            np.min(target_mito_coord[2]) - 2 : np.max(target_mito_coord[2]) + 2
        ],
        0
    )

    merged_imgs_tubular = np.where(
        merged_imgs_mito == 1,
        tubular_imgs[
            np.min(target_mito_coord[0]) - 2 : np.max(target_mito_coord[0]) + 2,
            np.min(target_mito_coord[1]) - 2 : np.max(target_mito_coord[1]) + 2,
            np.min(target_mito_coord[2]) - 2 : np.max(target_mito_coord[2]) + 2
        ],
        0
    )

    merged_imgs = np.concatenate(
        [merged_imgs_mito, merged_imgs_lamellar, merged_imgs_tubular],
        axis=3
    )
    
    out_dir =  f"Z:/paper_data/2021_cristae/correspose_PCA_without_angle/group1/shCtrl_cropped00{CROP_NO}_mito{str(num).zfill(3)}/imgs"
    os.makedirs(out_dir)

    for i in range(merged_imgs.shape[0]):
        cv2.imwrite(
            f"{out_dir}/{str(i).zfill(4)}.png",
            merged_imgs[i]
        )
    
    

del mito_imgs, lamellar_imgs, tubular_imgs, labeled_mito_imgs

In [ ]:
CROP_NO = 3


mito_imgs, _ =  load_Y_gray(f"Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_00{CROP_NO}/devided_mito/", normalize=False)
lamellar_imgs, _ = load_Y_gray(f"Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_00{CROP_NO}/merged_cristae_final/lamellar_with_postprocessing/", normalize=False)
tubular_imgs, _ = load_Y_gray(f"Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_00{CROP_NO}/merged_cristae_final/tubular_with_postprocessing/", normalize=False)

labeled_mito_imgs = cc3d.connected_components(mito_imgs[:,:,:,0].astype(int),  connectivity=6)


lst = list()
for node in group2:
    if node["Gene"] == "shCtrl" and node["cropped_No"] == CROP_NO:
        lst.append(node["mito_No"])
        
for num in lst:
    target_mito_coord = np.nonzero(labeled_mito_imgs==num)
    merged_imgs_mito = np.where(
        labeled_mito_imgs[
            np.min(target_mito_coord[0]) - 2 : np.max(target_mito_coord[0]) + 2,
            np.min(target_mito_coord[1]) - 2 : np.max(target_mito_coord[1]) + 2,
            np.min(target_mito_coord[2]) - 2 : np.max(target_mito_coord[2]) + 2
        ] == num,
        1,
        0
    )[..., np.newaxis]

    merged_imgs_lamellar = np.where(
        merged_imgs_mito == 1,
        lamellar_imgs[
            np.min(target_mito_coord[0]) - 2 : np.max(target_mito_coord[0]) + 2,
            np.min(target_mito_coord[1]) - 2 : np.max(target_mito_coord[1]) + 2,
            np.min(target_mito_coord[2]) - 2 : np.max(target_mito_coord[2]) + 2
        ],
        0
    )

    merged_imgs_tubular = np.where(
        merged_imgs_mito == 1,
        tubular_imgs[
            np.min(target_mito_coord[0]) - 2 : np.max(target_mito_coord[0]) + 2,
            np.min(target_mito_coord[1]) - 2 : np.max(target_mito_coord[1]) + 2,
            np.min(target_mito_coord[2]) - 2 : np.max(target_mito_coord[2]) + 2
        ],
        0
    )

    merged_imgs = np.concatenate(
        [merged_imgs_mito, merged_imgs_lamellar, merged_imgs_tubular],
        axis=3
    )
    
    out_dir =  f"Z:/paper_data/2021_cristae/correspose_PCA_without_angle/group1/shCtrl_cropped00{CROP_NO}_mito{str(num).zfill(3)}/imgs"
    os.makedirs(out_dir)

    for i in range(merged_imgs.shape[0]):
        cv2.imwrite(
            f"{out_dir}/{str(i).zfill(4)}.png",
            merged_imgs[i]
        )
    
    

del mito_imgs, lamellar_imgs, tubular_imgs, labeled_mito_imgs

In [ ]:
CROP_NO = 4


mito_imgs, _ =  load_Y_gray(f"Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_00{CROP_NO}/devided_mito/", normalize=False)
lamellar_imgs, _ = load_Y_gray(f"Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_00{CROP_NO}/merged_cristae_final/lamellar_with_postprocessing/", normalize=False)
tubular_imgs, _ = load_Y_gray(f"Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_00{CROP_NO}/merged_cristae_final/tubular_with_postprocessing/", normalize=False)

labeled_mito_imgs = cc3d.connected_components(mito_imgs[:,:,:,0].astype(int),  connectivity=6)


lst = list()
for node in group2:
    if node["Gene"] == "shCtrl" and node["cropped_No"] == CROP_NO:
        lst.append(node["mito_No"])
        
for num in lst:
    target_mito_coord = np.nonzero(labeled_mito_imgs==num)
    merged_imgs_mito = np.where(
        labeled_mito_imgs[
            np.min(target_mito_coord[0]) - 2 : np.max(target_mito_coord[0]) + 2,
            np.min(target_mito_coord[1]) - 2 : np.max(target_mito_coord[1]) + 2,
            np.min(target_mito_coord[2]) - 2 : np.max(target_mito_coord[2]) + 2
        ] == num,
        1,
        0
    )[..., np.newaxis]

    merged_imgs_lamellar = np.where(
        merged_imgs_mito == 1,
        lamellar_imgs[
            np.min(target_mito_coord[0]) - 2 : np.max(target_mito_coord[0]) + 2,
            np.min(target_mito_coord[1]) - 2 : np.max(target_mito_coord[1]) + 2,
            np.min(target_mito_coord[2]) - 2 : np.max(target_mito_coord[2]) + 2
        ],
        0
    )

    merged_imgs_tubular = np.where(
        merged_imgs_mito == 1,
        tubular_imgs[
            np.min(target_mito_coord[0]) - 2 : np.max(target_mito_coord[0]) + 2,
            np.min(target_mito_coord[1]) - 2 : np.max(target_mito_coord[1]) + 2,
            np.min(target_mito_coord[2]) - 2 : np.max(target_mito_coord[2]) + 2
        ],
        0
    )

    merged_imgs = np.concatenate(
        [merged_imgs_mito, merged_imgs_lamellar, merged_imgs_tubular],
        axis=3
    )
    
    out_dir =  f"Z:/paper_data/2021_cristae/correspose_PCA_without_angle/group1/shCtrl_cropped00{CROP_NO}_mito{str(num).zfill(3)}/imgs"
    os.makedirs(out_dir)

    for i in range(merged_imgs.shape[0]):
        cv2.imwrite(
            f"{out_dir}/{str(i).zfill(4)}.png",
            merged_imgs[i]
        )
    
    

del mito_imgs, lamellar_imgs, tubular_imgs, labeled_mito_imgs

In [ ]:
CROP_NO = 5


mito_imgs, _ =  load_Y_gray(f"Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_00{CROP_NO}/devided_mito/", normalize=False)
lamellar_imgs, _ = load_Y_gray(f"Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_00{CROP_NO}/merged_cristae_final/lamellar_with_postprocessing/", normalize=False)
tubular_imgs, _ = load_Y_gray(f"Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_00{CROP_NO}/merged_cristae_final/tubular_with_postprocessing/", normalize=False)

labeled_mito_imgs = cc3d.connected_components(mito_imgs[:,:,:,0].astype(int),  connectivity=6)


lst = list()
for node in group2:
    if node["Gene"] == "shCtrl" and node["cropped_No"] == CROP_NO:
        lst.append(node["mito_No"])
        
for num in lst:
    target_mito_coord = np.nonzero(labeled_mito_imgs==num)
    merged_imgs_mito = np.where(
        labeled_mito_imgs[
            np.min(target_mito_coord[0]) - 2 : np.max(target_mito_coord[0]) + 2,
            np.min(target_mito_coord[1]) - 2 : np.max(target_mito_coord[1]) + 2,
            np.min(target_mito_coord[2]) - 2 : np.max(target_mito_coord[2]) + 2
        ] == num,
        1,
        0
    )[..., np.newaxis]

    merged_imgs_lamellar = np.where(
        merged_imgs_mito == 1,
        lamellar_imgs[
            np.min(target_mito_coord[0]) - 2 : np.max(target_mito_coord[0]) + 2,
            np.min(target_mito_coord[1]) - 2 : np.max(target_mito_coord[1]) + 2,
            np.min(target_mito_coord[2]) - 2 : np.max(target_mito_coord[2]) + 2
        ],
        0
    )

    merged_imgs_tubular = np.where(
        merged_imgs_mito == 1,
        tubular_imgs[
            np.min(target_mito_coord[0]) - 2 : np.max(target_mito_coord[0]) + 2,
            np.min(target_mito_coord[1]) - 2 : np.max(target_mito_coord[1]) + 2,
            np.min(target_mito_coord[2]) - 2 : np.max(target_mito_coord[2]) + 2
        ],
        0
    )

    merged_imgs = np.concatenate(
        [merged_imgs_mito, merged_imgs_lamellar, merged_imgs_tubular],
        axis=3
    )
    
    out_dir =  f"Z:/paper_data/2021_cristae/correspose_PCA_without_angle/group1/shCtrl_cropped00{CROP_NO}_mito{str(num).zfill(3)}/imgs"
    os.makedirs(out_dir)

    for i in range(merged_imgs.shape[0]):
        cv2.imwrite(
            f"{out_dir}/{str(i).zfill(4)}.png",
            merged_imgs[i]
        )
    
    

del mito_imgs, lamellar_imgs, tubular_imgs, labeled_mito_imgs

In [ ]:
CROP_NO = 1


mito_imgs, _ =  load_Y_gray(f"Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_00{CROP_NO}/devided_mito/", normalize=False)
lamellar_imgs, _ = load_Y_gray(f"Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_00{CROP_NO}/merged_cristae_final/lamellar_with_postprocessing/", normalize=False)
tubular_imgs, _ = load_Y_gray(f"Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_00{CROP_NO}/merged_cristae_final/tubular_with_postprocessing/", normalize=False)

labeled_mito_imgs = cc3d.connected_components(mito_imgs[:,:,:,0].astype(int),  connectivity=6)


lst = list()
for node in group2:
    if node["Gene"] == "shOPA1" and node["cropped_No"] == CROP_NO:
        lst.append(node["mito_No"])
        
for num in lst:
    target_mito_coord = np.nonzero(labeled_mito_imgs==num)
    merged_imgs_mito = np.where(
        labeled_mito_imgs[
            np.min(target_mito_coord[0]) - 2 : np.max(target_mito_coord[0]) + 2,
            np.min(target_mito_coord[1]) - 2 : np.max(target_mito_coord[1]) + 2,
            np.min(target_mito_coord[2]) - 2 : np.max(target_mito_coord[2]) + 2
        ] == num,
        1,
        0
    )[..., np.newaxis]

    merged_imgs_lamellar = np.where(
        merged_imgs_mito == 1,
        lamellar_imgs[
            np.min(target_mito_coord[0]) - 2 : np.max(target_mito_coord[0]) + 2,
            np.min(target_mito_coord[1]) - 2 : np.max(target_mito_coord[1]) + 2,
            np.min(target_mito_coord[2]) - 2 : np.max(target_mito_coord[2]) + 2
        ],
        0
    )

    merged_imgs_tubular = np.where(
        merged_imgs_mito == 1,
        tubular_imgs[
            np.min(target_mito_coord[0]) - 2 : np.max(target_mito_coord[0]) + 2,
            np.min(target_mito_coord[1]) - 2 : np.max(target_mito_coord[1]) + 2,
            np.min(target_mito_coord[2]) - 2 : np.max(target_mito_coord[2]) + 2
        ],
        0
    )

    merged_imgs = np.concatenate(
        [merged_imgs_mito, merged_imgs_lamellar, merged_imgs_tubular],
        axis=3
    )
    
    out_dir =  f"Z:/paper_data/2021_cristae/correspose_PCA_without_angle/group1/shOPA1_cropped00{CROP_NO}_mito{str(num).zfill(3)}/imgs"
    os.makedirs(out_dir)

    for i in range(merged_imgs.shape[0]):
        cv2.imwrite(
            f"{out_dir}/{str(i).zfill(4)}.png",
            merged_imgs[i]
        )
    
    

del mito_imgs, lamellar_imgs, tubular_imgs, labeled_mito_imgs

In [ ]:
CROP_NO = 2


mito_imgs, _ =  load_Y_gray(f"Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_00{CROP_NO}/devided_mito/", normalize=False)
lamellar_imgs, _ = load_Y_gray(f"Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_00{CROP_NO}/merged_cristae_final/lamellar_with_postprocessing/", normalize=False)
tubular_imgs, _ = load_Y_gray(f"Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_00{CROP_NO}/merged_cristae_final/tubular_with_postprocessing/", normalize=False)

labeled_mito_imgs = cc3d.connected_components(mito_imgs[:,:,:,0].astype(int),  connectivity=6)


lst = list()
for node in group2:
    if node["Gene"] == "shOPA1" and node["cropped_No"] == CROP_NO:
        lst.append(node["mito_No"])
        
for num in lst:
    target_mito_coord = np.nonzero(labeled_mito_imgs==num)
    merged_imgs_mito = np.where(
        labeled_mito_imgs[
            np.min(target_mito_coord[0]) - 2 : np.max(target_mito_coord[0]) + 2,
            np.min(target_mito_coord[1]) - 2 : np.max(target_mito_coord[1]) + 2,
            np.min(target_mito_coord[2]) - 2 : np.max(target_mito_coord[2]) + 2
        ] == num,
        1,
        0
    )[..., np.newaxis]

    merged_imgs_lamellar = np.where(
        merged_imgs_mito == 1,
        lamellar_imgs[
            np.min(target_mito_coord[0]) - 2 : np.max(target_mito_coord[0]) + 2,
            np.min(target_mito_coord[1]) - 2 : np.max(target_mito_coord[1]) + 2,
            np.min(target_mito_coord[2]) - 2 : np.max(target_mito_coord[2]) + 2
        ],
        0
    )

    merged_imgs_tubular = np.where(
        merged_imgs_mito == 1,
        tubular_imgs[
            np.min(target_mito_coord[0]) - 2 : np.max(target_mito_coord[0]) + 2,
            np.min(target_mito_coord[1]) - 2 : np.max(target_mito_coord[1]) + 2,
            np.min(target_mito_coord[2]) - 2 : np.max(target_mito_coord[2]) + 2
        ],
        0
    )

    merged_imgs = np.concatenate(
        [merged_imgs_mito, merged_imgs_lamellar, merged_imgs_tubular],
        axis=3
    )
    
    out_dir =  f"Z:/paper_data/2021_cristae/correspose_PCA_without_angle/group1/shOPA1_cropped00{CROP_NO}_mito{str(num).zfill(3)}/imgs"
    os.makedirs(out_dir)

    for i in range(merged_imgs.shape[0]):
        cv2.imwrite(
            f"{out_dir}/{str(i).zfill(4)}.png",
            merged_imgs[i]
        )
    
    

del mito_imgs, lamellar_imgs, tubular_imgs, labeled_mito_imgs

In [ ]:
CROP_NO = 3


mito_imgs, _ =  load_Y_gray(f"Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_00{CROP_NO}/devided_mito/", normalize=False)
lamellar_imgs, _ = load_Y_gray(f"Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_00{CROP_NO}/merged_cristae_final/lamellar_with_postprocessing/", normalize=False)
tubular_imgs, _ = load_Y_gray(f"Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_00{CROP_NO}/merged_cristae_final/tubular_with_postprocessing/", normalize=False)

labeled_mito_imgs = cc3d.connected_components(mito_imgs[:,:,:,0].astype(int),  connectivity=6)


lst = list()
for node in group2:
    if node["Gene"] == "shOPA1" and node["cropped_No"] == CROP_NO:
        lst.append(node["mito_No"])
        
for num in lst:
    target_mito_coord = np.nonzero(labeled_mito_imgs==num)
    merged_imgs_mito = np.where(
        labeled_mito_imgs[
            np.min(target_mito_coord[0]) - 2 : np.max(target_mito_coord[0]) + 2,
            np.min(target_mito_coord[1]) - 2 : np.max(target_mito_coord[1]) + 2,
            np.min(target_mito_coord[2]) - 2 : np.max(target_mito_coord[2]) + 2
        ] == num,
        1,
        0
    )[..., np.newaxis]

    merged_imgs_lamellar = np.where(
        merged_imgs_mito == 1,
        lamellar_imgs[
            np.min(target_mito_coord[0]) - 2 : np.max(target_mito_coord[0]) + 2,
            np.min(target_mito_coord[1]) - 2 : np.max(target_mito_coord[1]) + 2,
            np.min(target_mito_coord[2]) - 2 : np.max(target_mito_coord[2]) + 2
        ],
        0
    )

    merged_imgs_tubular = np.where(
        merged_imgs_mito == 1,
        tubular_imgs[
            np.min(target_mito_coord[0]) - 2 : np.max(target_mito_coord[0]) + 2,
            np.min(target_mito_coord[1]) - 2 : np.max(target_mito_coord[1]) + 2,
            np.min(target_mito_coord[2]) - 2 : np.max(target_mito_coord[2]) + 2
        ],
        0
    )

    merged_imgs = np.concatenate(
        [merged_imgs_mito, merged_imgs_lamellar, merged_imgs_tubular],
        axis=3
    )
    
    out_dir =  f"Z:/paper_data/2021_cristae/correspose_PCA_without_angle/group1/shOPA1_cropped00{CROP_NO}_mito{str(num).zfill(3)}/imgs"
    os.makedirs(out_dir)

    for i in range(merged_imgs.shape[0]):
        cv2.imwrite(
            f"{out_dir}/{str(i).zfill(4)}.png",
            merged_imgs[i]
        )
    
    

del mito_imgs, lamellar_imgs, tubular_imgs, labeled_mito_imgs

In [ ]:
CROP_NO = 4


mito_imgs, _ =  load_Y_gray(f"Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_00{CROP_NO}/devided_mito/", normalize=False)
lamellar_imgs, _ = load_Y_gray(f"Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_00{CROP_NO}/merged_cristae_final/lamellar_with_postprocessing/", normalize=False)
tubular_imgs, _ = load_Y_gray(f"Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_00{CROP_NO}/merged_cristae_final/tubular_with_postprocessing/", normalize=False)

labeled_mito_imgs = cc3d.connected_components(mito_imgs[:,:,:,0].astype(int),  connectivity=6)


lst = list()
for node in group2:
    if node["Gene"] == "shOPA1" and node["cropped_No"] == CROP_NO:
        lst.append(node["mito_No"])
        
for num in lst:
    target_mito_coord = np.nonzero(labeled_mito_imgs==num)
    merged_imgs_mito = np.where(
        labeled_mito_imgs[
            np.min(target_mito_coord[0]) - 2 : np.max(target_mito_coord[0]) + 2,
            np.min(target_mito_coord[1]) - 2 : np.max(target_mito_coord[1]) + 2,
            np.min(target_mito_coord[2]) - 2 : np.max(target_mito_coord[2]) + 2
        ] == num,
        1,
        0
    )[..., np.newaxis]

    merged_imgs_lamellar = np.where(
        merged_imgs_mito == 1,
        lamellar_imgs[
            np.min(target_mito_coord[0]) - 2 : np.max(target_mito_coord[0]) + 2,
            np.min(target_mito_coord[1]) - 2 : np.max(target_mito_coord[1]) + 2,
            np.min(target_mito_coord[2]) - 2 : np.max(target_mito_coord[2]) + 2
        ],
        0
    )

    merged_imgs_tubular = np.where(
        merged_imgs_mito == 1,
        tubular_imgs[
            np.min(target_mito_coord[0]) - 2 : np.max(target_mito_coord[0]) + 2,
            np.min(target_mito_coord[1]) - 2 : np.max(target_mito_coord[1]) + 2,
            np.min(target_mito_coord[2]) - 2 : np.max(target_mito_coord[2]) + 2
        ],
        0
    )

    merged_imgs = np.concatenate(
        [merged_imgs_mito, merged_imgs_lamellar, merged_imgs_tubular],
        axis=3
    )
    
    out_dir =  f"Z:/paper_data/2021_cristae/correspose_PCA_without_angle/group1/shOPA1_cropped00{CROP_NO}_mito{str(num).zfill(3)}/imgs"
    os.makedirs(out_dir)

    for i in range(merged_imgs.shape[0]):
        cv2.imwrite(
            f"{out_dir}/{str(i).zfill(4)}.png",
            merged_imgs[i]
        )
    
    

del mito_imgs, lamellar_imgs, tubular_imgs, labeled_mito_imgs

In [ ]:
CROP_NO = 5


mito_imgs, _ =  load_Y_gray(f"Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_00{CROP_NO}/devided_mito/", normalize=False)
lamellar_imgs, _ = load_Y_gray(f"Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_00{CROP_NO}/merged_cristae_final/lamellar_with_postprocessing/", normalize=False)
tubular_imgs, _ = load_Y_gray(f"Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_00{CROP_NO}/merged_cristae_final/tubular_with_postprocessing/", normalize=False)

labeled_mito_imgs = cc3d.connected_components(mito_imgs[:,:,:,0].astype(int),  connectivity=6)


lst = list()
for node in group2:
    if node["Gene"] == "shOPA1" and node["cropped_No"] == CROP_NO:
        lst.append(node["mito_No"])
        
for num in lst:
    target_mito_coord = np.nonzero(labeled_mito_imgs==num)
    merged_imgs_mito = np.where(
        labeled_mito_imgs[
            np.min(target_mito_coord[0]) - 2 : np.max(target_mito_coord[0]) + 2,
            np.min(target_mito_coord[1]) - 2 : np.max(target_mito_coord[1]) + 2,
            np.min(target_mito_coord[2]) - 2 : np.max(target_mito_coord[2]) + 2
        ] == num,
        1,
        0
    )[..., np.newaxis]

    merged_imgs_lamellar = np.where(
        merged_imgs_mito == 1,
        lamellar_imgs[
            np.min(target_mito_coord[0]) - 2 : np.max(target_mito_coord[0]) + 2,
            np.min(target_mito_coord[1]) - 2 : np.max(target_mito_coord[1]) + 2,
            np.min(target_mito_coord[2]) - 2 : np.max(target_mito_coord[2]) + 2
        ],
        0
    )

    merged_imgs_tubular = np.where(
        merged_imgs_mito == 1,
        tubular_imgs[
            np.min(target_mito_coord[0]) - 2 : np.max(target_mito_coord[0]) + 2,
            np.min(target_mito_coord[1]) - 2 : np.max(target_mito_coord[1]) + 2,
            np.min(target_mito_coord[2]) - 2 : np.max(target_mito_coord[2]) + 2
        ],
        0
    )

    merged_imgs = np.concatenate(
        [merged_imgs_mito, merged_imgs_lamellar, merged_imgs_tubular],
        axis=3
    )
    
    out_dir =  f"Z:/paper_data/2021_cristae/correspose_PCA_without_angle/group1/shOPA1_cropped00{CROP_NO}_mito{str(num).zfill(3)}/imgs"
    os.makedirs(out_dir)

    for i in range(merged_imgs.shape[0]):
        cv2.imwrite(
            f"{out_dir}/{str(i).zfill(4)}.png",
            merged_imgs[i]
        )
    
    

del mito_imgs, lamellar_imgs, tubular_imgs, labeled_mito_imgs

In [ ]:
CROP_NO = 1


mito_imgs, _ =  load_Y_gray(f"Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_00{CROP_NO}/devided_mito/", normalize=False)
lamellar_imgs, _ = load_Y_gray(f"Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_00{CROP_NO}/merged_cristae_final/lamellar_with_postprocessing/", normalize=False)
tubular_imgs, _ = load_Y_gray(f"Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_00{CROP_NO}/merged_cristae_final/tubular_with_postprocessing/", normalize=False)

labeled_mito_imgs = cc3d.connected_components(mito_imgs[:,:,:,0].astype(int),  connectivity=6)


lst = list()
for node in group1:
    if node["Gene"] == "shCtrl" and node["cropped_No"] == CROP_NO:
        lst.append(node["mito_No"])
        
for num in lst:
    target_mito_coord = np.nonzero(labeled_mito_imgs==num)
    merged_imgs_mito = np.where(
        labeled_mito_imgs[
            np.min(target_mito_coord[0]) - 2 : np.max(target_mito_coord[0]) + 2,
            np.min(target_mito_coord[1]) - 2 : np.max(target_mito_coord[1]) + 2,
            np.min(target_mito_coord[2]) - 2 : np.max(target_mito_coord[2]) + 2
        ] == num,
        1,
        0
    )[..., np.newaxis]

    merged_imgs_lamellar = np.where(
        merged_imgs_mito == 1,
        lamellar_imgs[
            np.min(target_mito_coord[0]) - 2 : np.max(target_mito_coord[0]) + 2,
            np.min(target_mito_coord[1]) - 2 : np.max(target_mito_coord[1]) + 2,
            np.min(target_mito_coord[2]) - 2 : np.max(target_mito_coord[2]) + 2
        ],
        0
    )

    merged_imgs_tubular = np.where(
        merged_imgs_mito == 1,
        tubular_imgs[
            np.min(target_mito_coord[0]) - 2 : np.max(target_mito_coord[0]) + 2,
            np.min(target_mito_coord[1]) - 2 : np.max(target_mito_coord[1]) + 2,
            np.min(target_mito_coord[2]) - 2 : np.max(target_mito_coord[2]) + 2
        ],
        0
    )

    merged_imgs = np.concatenate(
        [merged_imgs_mito, merged_imgs_lamellar, merged_imgs_tubular],
        axis=3
    )
    
    out_dir =  f"Z:/paper_data/2021_cristae/correspose_PCA_without_angle/group2/shCtrl_cropped00{CROP_NO}_mito{str(num).zfill(3)}/imgs"
    os.makedirs(out_dir)

    for i in range(merged_imgs.shape[0]):
        cv2.imwrite(
            f"{out_dir}/{str(i).zfill(4)}.png",
            merged_imgs[i]
        )
    
    

del mito_imgs, lamellar_imgs, tubular_imgs, labeled_mito_imgs

In [ ]:
CROP_NO = 2


mito_imgs, _ =  load_Y_gray(f"Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_00{CROP_NO}/devided_mito/", normalize=False)
lamellar_imgs, _ = load_Y_gray(f"Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_00{CROP_NO}/merged_cristae_final/lamellar_with_postprocessing/", normalize=False)
tubular_imgs, _ = load_Y_gray(f"Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_00{CROP_NO}/merged_cristae_final/tubular_with_postprocessing/", normalize=False)

labeled_mito_imgs = cc3d.connected_components(mito_imgs[:,:,:,0].astype(int),  connectivity=6)


lst = list()
for node in group1:
    if node["Gene"] == "shCtrl" and node["cropped_No"] == CROP_NO:
        lst.append(node["mito_No"])
        
for num in lst:
    target_mito_coord = np.nonzero(labeled_mito_imgs==num)
    merged_imgs_mito = np.where(
        labeled_mito_imgs[
            np.min(target_mito_coord[0]) - 2 : np.max(target_mito_coord[0]) + 2,
            np.min(target_mito_coord[1]) - 2 : np.max(target_mito_coord[1]) + 2,
            np.min(target_mito_coord[2]) - 2 : np.max(target_mito_coord[2]) + 2
        ] == num,
        1,
        0
    )[..., np.newaxis]

    merged_imgs_lamellar = np.where(
        merged_imgs_mito == 1,
        lamellar_imgs[
            np.min(target_mito_coord[0]) - 2 : np.max(target_mito_coord[0]) + 2,
            np.min(target_mito_coord[1]) - 2 : np.max(target_mito_coord[1]) + 2,
            np.min(target_mito_coord[2]) - 2 : np.max(target_mito_coord[2]) + 2
        ],
        0
    )

    merged_imgs_tubular = np.where(
        merged_imgs_mito == 1,
        tubular_imgs[
            np.min(target_mito_coord[0]) - 2 : np.max(target_mito_coord[0]) + 2,
            np.min(target_mito_coord[1]) - 2 : np.max(target_mito_coord[1]) + 2,
            np.min(target_mito_coord[2]) - 2 : np.max(target_mito_coord[2]) + 2
        ],
        0
    )

    merged_imgs = np.concatenate(
        [merged_imgs_mito, merged_imgs_lamellar, merged_imgs_tubular],
        axis=3
    )
    
    out_dir =  f"Z:/paper_data/2021_cristae/correspose_PCA_without_angle/group2/shCtrl_cropped00{CROP_NO}_mito{str(num).zfill(3)}/imgs"
    os.makedirs(out_dir)

    for i in range(merged_imgs.shape[0]):
        cv2.imwrite(
            f"{out_dir}/{str(i).zfill(4)}.png",
            merged_imgs[i]
        )
    
    

del mito_imgs, lamellar_imgs, tubular_imgs, labeled_mito_imgs

In [ ]:
CROP_NO = 3


mito_imgs, _ =  load_Y_gray(f"Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_00{CROP_NO}/devided_mito/", normalize=False)
lamellar_imgs, _ = load_Y_gray(f"Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_00{CROP_NO}/merged_cristae_final/lamellar_with_postprocessing/", normalize=False)
tubular_imgs, _ = load_Y_gray(f"Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_00{CROP_NO}/merged_cristae_final/tubular_with_postprocessing/", normalize=False)

labeled_mito_imgs = cc3d.connected_components(mito_imgs[:,:,:,0].astype(int),  connectivity=6)


lst = list()
for node in group1:
    if node["Gene"] == "shCtrl" and node["cropped_No"] == CROP_NO:
        lst.append(node["mito_No"])
        
for num in lst:
    target_mito_coord = np.nonzero(labeled_mito_imgs==num)
    merged_imgs_mito = np.where(
        labeled_mito_imgs[
            np.min(target_mito_coord[0]) - 2 : np.max(target_mito_coord[0]) + 2,
            np.min(target_mito_coord[1]) - 2 : np.max(target_mito_coord[1]) + 2,
            np.min(target_mito_coord[2]) - 2 : np.max(target_mito_coord[2]) + 2
        ] == num,
        1,
        0
    )[..., np.newaxis]

    merged_imgs_lamellar = np.where(
        merged_imgs_mito == 1,
        lamellar_imgs[
            np.min(target_mito_coord[0]) - 2 : np.max(target_mito_coord[0]) + 2,
            np.min(target_mito_coord[1]) - 2 : np.max(target_mito_coord[1]) + 2,
            np.min(target_mito_coord[2]) - 2 : np.max(target_mito_coord[2]) + 2
        ],
        0
    )

    merged_imgs_tubular = np.where(
        merged_imgs_mito == 1,
        tubular_imgs[
            np.min(target_mito_coord[0]) - 2 : np.max(target_mito_coord[0]) + 2,
            np.min(target_mito_coord[1]) - 2 : np.max(target_mito_coord[1]) + 2,
            np.min(target_mito_coord[2]) - 2 : np.max(target_mito_coord[2]) + 2
        ],
        0
    )

    merged_imgs = np.concatenate(
        [merged_imgs_mito, merged_imgs_lamellar, merged_imgs_tubular],
        axis=3
    )
    
    out_dir =  f"Z:/paper_data/2021_cristae/correspose_PCA_without_angle/group2/shCtrl_cropped00{CROP_NO}_mito{str(num).zfill(3)}/imgs"
    os.makedirs(out_dir)

    for i in range(merged_imgs.shape[0]):
        cv2.imwrite(
            f"{out_dir}/{str(i).zfill(4)}.png",
            merged_imgs[i]
        )
    
    

del mito_imgs, lamellar_imgs, tubular_imgs, labeled_mito_imgs

In [ ]:
CROP_NO = 4


mito_imgs, _ =  load_Y_gray(f"Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_00{CROP_NO}/devided_mito/", normalize=False)
lamellar_imgs, _ = load_Y_gray(f"Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_00{CROP_NO}/merged_cristae_final/lamellar_with_postprocessing/", normalize=False)
tubular_imgs, _ = load_Y_gray(f"Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_00{CROP_NO}/merged_cristae_final/tubular_with_postprocessing/", normalize=False)

labeled_mito_imgs = cc3d.connected_components(mito_imgs[:,:,:,0].astype(int),  connectivity=6)


lst = list()
for node in group1:
    if node["Gene"] == "shCtrl" and node["cropped_No"] == CROP_NO:
        lst.append(node["mito_No"])
        
for num in lst:
    target_mito_coord = np.nonzero(labeled_mito_imgs==num)
    merged_imgs_mito = np.where(
        labeled_mito_imgs[
            np.min(target_mito_coord[0]) - 2 : np.max(target_mito_coord[0]) + 2,
            np.min(target_mito_coord[1]) - 2 : np.max(target_mito_coord[1]) + 2,
            np.min(target_mito_coord[2]) - 2 : np.max(target_mito_coord[2]) + 2
        ] == num,
        1,
        0
    )[..., np.newaxis]

    merged_imgs_lamellar = np.where(
        merged_imgs_mito == 1,
        lamellar_imgs[
            np.min(target_mito_coord[0]) - 2 : np.max(target_mito_coord[0]) + 2,
            np.min(target_mito_coord[1]) - 2 : np.max(target_mito_coord[1]) + 2,
            np.min(target_mito_coord[2]) - 2 : np.max(target_mito_coord[2]) + 2
        ],
        0
    )

    merged_imgs_tubular = np.where(
        merged_imgs_mito == 1,
        tubular_imgs[
            np.min(target_mito_coord[0]) - 2 : np.max(target_mito_coord[0]) + 2,
            np.min(target_mito_coord[1]) - 2 : np.max(target_mito_coord[1]) + 2,
            np.min(target_mito_coord[2]) - 2 : np.max(target_mito_coord[2]) + 2
        ],
        0
    )

    merged_imgs = np.concatenate(
        [merged_imgs_mito, merged_imgs_lamellar, merged_imgs_tubular],
        axis=3
    )
    
    out_dir =  f"Z:/paper_data/2021_cristae/correspose_PCA_without_angle/group2/shCtrl_cropped00{CROP_NO}_mito{str(num).zfill(3)}/imgs"
    os.makedirs(out_dir)

    for i in range(merged_imgs.shape[0]):
        cv2.imwrite(
            f"{out_dir}/{str(i).zfill(4)}.png",
            merged_imgs[i]
        )
    
    

del mito_imgs, lamellar_imgs, tubular_imgs, labeled_mito_imgs

In [ ]:
CROP_NO = 5


mito_imgs, _ =  load_Y_gray(f"Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_00{CROP_NO}/devided_mito/", normalize=False)
lamellar_imgs, _ = load_Y_gray(f"Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_00{CROP_NO}/merged_cristae_final/lamellar_with_postprocessing/", normalize=False)
tubular_imgs, _ = load_Y_gray(f"Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_00{CROP_NO}/merged_cristae_final/tubular_with_postprocessing/", normalize=False)

labeled_mito_imgs = cc3d.connected_components(mito_imgs[:,:,:,0].astype(int),  connectivity=6)


lst = list()
for node in group1:
    if node["Gene"] == "shCtrl" and node["cropped_No"] == CROP_NO:
        lst.append(node["mito_No"])
        
for num in lst:
    target_mito_coord = np.nonzero(labeled_mito_imgs==num)
    merged_imgs_mito = np.where(
        labeled_mito_imgs[
            np.min(target_mito_coord[0]) - 2 : np.max(target_mito_coord[0]) + 2,
            np.min(target_mito_coord[1]) - 2 : np.max(target_mito_coord[1]) + 2,
            np.min(target_mito_coord[2]) - 2 : np.max(target_mito_coord[2]) + 2
        ] == num,
        1,
        0
    )[..., np.newaxis]

    merged_imgs_lamellar = np.where(
        merged_imgs_mito == 1,
        lamellar_imgs[
            np.min(target_mito_coord[0]) - 2 : np.max(target_mito_coord[0]) + 2,
            np.min(target_mito_coord[1]) - 2 : np.max(target_mito_coord[1]) + 2,
            np.min(target_mito_coord[2]) - 2 : np.max(target_mito_coord[2]) + 2
        ],
        0
    )

    merged_imgs_tubular = np.where(
        merged_imgs_mito == 1,
        tubular_imgs[
            np.min(target_mito_coord[0]) - 2 : np.max(target_mito_coord[0]) + 2,
            np.min(target_mito_coord[1]) - 2 : np.max(target_mito_coord[1]) + 2,
            np.min(target_mito_coord[2]) - 2 : np.max(target_mito_coord[2]) + 2
        ],
        0
    )

    merged_imgs = np.concatenate(
        [merged_imgs_mito, merged_imgs_lamellar, merged_imgs_tubular],
        axis=3
    )
    
    out_dir =  f"Z:/paper_data/2021_cristae/correspose_PCA_without_angle/group2/shCtrl_cropped00{CROP_NO}_mito{str(num).zfill(3)}/imgs"
    os.makedirs(out_dir)

    for i in range(merged_imgs.shape[0]):
        cv2.imwrite(
            f"{out_dir}/{str(i).zfill(4)}.png",
            merged_imgs[i]
        )
    
    

del mito_imgs, lamellar_imgs, tubular_imgs, labeled_mito_imgs

In [ ]:
CROP_NO = 1


mito_imgs, _ =  load_Y_gray(f"Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_00{CROP_NO}/devided_mito/", normalize=False)
lamellar_imgs, _ = load_Y_gray(f"Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_00{CROP_NO}/merged_cristae_final/lamellar_with_postprocessing/", normalize=False)
tubular_imgs, _ = load_Y_gray(f"Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_00{CROP_NO}/merged_cristae_final/tubular_with_postprocessing/", normalize=False)

labeled_mito_imgs = cc3d.connected_components(mito_imgs[:,:,:,0].astype(int),  connectivity=6)


lst = list()
for node in group1:
    if node["Gene"] == "shOPA1" and node["cropped_No"] == CROP_NO:
        lst.append(node["mito_No"])
        
for num in lst:
    target_mito_coord = np.nonzero(labeled_mito_imgs==num)
    merged_imgs_mito = np.where(
        labeled_mito_imgs[
            np.min(target_mito_coord[0]) - 2 : np.max(target_mito_coord[0]) + 2,
            np.min(target_mito_coord[1]) - 2 : np.max(target_mito_coord[1]) + 2,
            np.min(target_mito_coord[2]) - 2 : np.max(target_mito_coord[2]) + 2
        ] == num,
        1,
        0
    )[..., np.newaxis]

    merged_imgs_lamellar = np.where(
        merged_imgs_mito == 1,
        lamellar_imgs[
            np.min(target_mito_coord[0]) - 2 : np.max(target_mito_coord[0]) + 2,
            np.min(target_mito_coord[1]) - 2 : np.max(target_mito_coord[1]) + 2,
            np.min(target_mito_coord[2]) - 2 : np.max(target_mito_coord[2]) + 2
        ],
        0
    )

    merged_imgs_tubular = np.where(
        merged_imgs_mito == 1,
        tubular_imgs[
            np.min(target_mito_coord[0]) - 2 : np.max(target_mito_coord[0]) + 2,
            np.min(target_mito_coord[1]) - 2 : np.max(target_mito_coord[1]) + 2,
            np.min(target_mito_coord[2]) - 2 : np.max(target_mito_coord[2]) + 2
        ],
        0
    )

    merged_imgs = np.concatenate(
        [merged_imgs_mito, merged_imgs_lamellar, merged_imgs_tubular],
        axis=3
    )
    
    out_dir =  f"Z:/paper_data/2021_cristae/correspose_PCA_without_angle/group2/shOPA1_cropped00{CROP_NO}_mito{str(num).zfill(3)}/imgs"
    os.makedirs(out_dir)

    for i in range(merged_imgs.shape[0]):
        cv2.imwrite(
            f"{out_dir}/{str(i).zfill(4)}.png",
            merged_imgs[i]
        )
    
    

del mito_imgs, lamellar_imgs, tubular_imgs, labeled_mito_imgs

In [ ]:
CROP_NO = 2


mito_imgs, _ =  load_Y_gray(f"Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_00{CROP_NO}/devided_mito/", normalize=False)
lamellar_imgs, _ = load_Y_gray(f"Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_00{CROP_NO}/merged_cristae_final/lamellar_with_postprocessing/", normalize=False)
tubular_imgs, _ = load_Y_gray(f"Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_00{CROP_NO}/merged_cristae_final/tubular_with_postprocessing/", normalize=False)

labeled_mito_imgs = cc3d.connected_components(mito_imgs[:,:,:,0].astype(int),  connectivity=6)


lst = list()
for node in group1:
    if node["Gene"] == "shOPA1" and node["cropped_No"] == CROP_NO:
        lst.append(node["mito_No"])
        
for num in lst:
    target_mito_coord = np.nonzero(labeled_mito_imgs==num)
    merged_imgs_mito = np.where(
        labeled_mito_imgs[
            np.min(target_mito_coord[0]) - 2 : np.max(target_mito_coord[0]) + 2,
            np.min(target_mito_coord[1]) - 2 : np.max(target_mito_coord[1]) + 2,
            np.min(target_mito_coord[2]) - 2 : np.max(target_mito_coord[2]) + 2
        ] == num,
        1,
        0
    )[..., np.newaxis]

    merged_imgs_lamellar = np.where(
        merged_imgs_mito == 1,
        lamellar_imgs[
            np.min(target_mito_coord[0]) - 2 : np.max(target_mito_coord[0]) + 2,
            np.min(target_mito_coord[1]) - 2 : np.max(target_mito_coord[1]) + 2,
            np.min(target_mito_coord[2]) - 2 : np.max(target_mito_coord[2]) + 2
        ],
        0
    )

    merged_imgs_tubular = np.where(
        merged_imgs_mito == 1,
        tubular_imgs[
            np.min(target_mito_coord[0]) - 2 : np.max(target_mito_coord[0]) + 2,
            np.min(target_mito_coord[1]) - 2 : np.max(target_mito_coord[1]) + 2,
            np.min(target_mito_coord[2]) - 2 : np.max(target_mito_coord[2]) + 2
        ],
        0
    )

    merged_imgs = np.concatenate(
        [merged_imgs_mito, merged_imgs_lamellar, merged_imgs_tubular],
        axis=3
    )
    
    out_dir =  f"Z:/paper_data/2021_cristae/correspose_PCA_without_angle/group2/shOPA1_cropped00{CROP_NO}_mito{str(num).zfill(3)}/imgs"
    os.makedirs(out_dir)

    for i in range(merged_imgs.shape[0]):
        cv2.imwrite(
            f"{out_dir}/{str(i).zfill(4)}.png",
            merged_imgs[i]
        )
    
    

del mito_imgs, lamellar_imgs, tubular_imgs, labeled_mito_imgs

In [ ]:
CROP_NO = 3


mito_imgs, _ =  load_Y_gray(f"Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_00{CROP_NO}/devided_mito/", normalize=False)
lamellar_imgs, _ = load_Y_gray(f"Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_00{CROP_NO}/merged_cristae_final/lamellar_with_postprocessing/", normalize=False)
tubular_imgs, _ = load_Y_gray(f"Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_00{CROP_NO}/merged_cristae_final/tubular_with_postprocessing/", normalize=False)

labeled_mito_imgs = cc3d.connected_components(mito_imgs[:,:,:,0].astype(int),  connectivity=6)


lst = list()
for node in group1:
    if node["Gene"] == "shOPA1" and node["cropped_No"] == CROP_NO:
        lst.append(node["mito_No"])
        
for num in lst:
    target_mito_coord = np.nonzero(labeled_mito_imgs==num)
    merged_imgs_mito = np.where(
        labeled_mito_imgs[
            np.min(target_mito_coord[0]) - 2 : np.max(target_mito_coord[0]) + 2,
            np.min(target_mito_coord[1]) - 2 : np.max(target_mito_coord[1]) + 2,
            np.min(target_mito_coord[2]) - 2 : np.max(target_mito_coord[2]) + 2
        ] == num,
        1,
        0
    )[..., np.newaxis]

    merged_imgs_lamellar = np.where(
        merged_imgs_mito == 1,
        lamellar_imgs[
            np.min(target_mito_coord[0]) - 2 : np.max(target_mito_coord[0]) + 2,
            np.min(target_mito_coord[1]) - 2 : np.max(target_mito_coord[1]) + 2,
            np.min(target_mito_coord[2]) - 2 : np.max(target_mito_coord[2]) + 2
        ],
        0
    )

    merged_imgs_tubular = np.where(
        merged_imgs_mito == 1,
        tubular_imgs[
            np.min(target_mito_coord[0]) - 2 : np.max(target_mito_coord[0]) + 2,
            np.min(target_mito_coord[1]) - 2 : np.max(target_mito_coord[1]) + 2,
            np.min(target_mito_coord[2]) - 2 : np.max(target_mito_coord[2]) + 2
        ],
        0
    )

    merged_imgs = np.concatenate(
        [merged_imgs_mito, merged_imgs_lamellar, merged_imgs_tubular],
        axis=3
    )
    
    out_dir =  f"Z:/paper_data/2021_cristae/correspose_PCA_without_angle/group2/shOPA1_cropped00{CROP_NO}_mito{str(num).zfill(3)}/imgs"
    os.makedirs(out_dir)

    for i in range(merged_imgs.shape[0]):
        cv2.imwrite(
            f"{out_dir}/{str(i).zfill(4)}.png",
            merged_imgs[i]
        )
    
    

del mito_imgs, lamellar_imgs, tubular_imgs, labeled_mito_imgs

In [ ]:
CROP_NO = 4


mito_imgs, _ =  load_Y_gray(f"Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_00{CROP_NO}/devided_mito/", normalize=False)
lamellar_imgs, _ = load_Y_gray(f"Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_00{CROP_NO}/merged_cristae_final/lamellar_with_postprocessing/", normalize=False)
tubular_imgs, _ = load_Y_gray(f"Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_00{CROP_NO}/merged_cristae_final/tubular_with_postprocessing/", normalize=False)

labeled_mito_imgs = cc3d.connected_components(mito_imgs[:,:,:,0].astype(int),  connectivity=6)


lst = list()
for node in group1:
    if node["Gene"] == "shOPA1" and node["cropped_No"] == CROP_NO:
        lst.append(node["mito_No"])
        
for num in lst:
    target_mito_coord = np.nonzero(labeled_mito_imgs==num)
    merged_imgs_mito = np.where(
        labeled_mito_imgs[
            np.min(target_mito_coord[0]) - 2 : np.max(target_mito_coord[0]) + 2,
            np.min(target_mito_coord[1]) - 2 : np.max(target_mito_coord[1]) + 2,
            np.min(target_mito_coord[2]) - 2 : np.max(target_mito_coord[2]) + 2
        ] == num,
        1,
        0
    )[..., np.newaxis]

    merged_imgs_lamellar = np.where(
        merged_imgs_mito == 1,
        lamellar_imgs[
            np.min(target_mito_coord[0]) - 2 : np.max(target_mito_coord[0]) + 2,
            np.min(target_mito_coord[1]) - 2 : np.max(target_mito_coord[1]) + 2,
            np.min(target_mito_coord[2]) - 2 : np.max(target_mito_coord[2]) + 2
        ],
        0
    )

    merged_imgs_tubular = np.where(
        merged_imgs_mito == 1,
        tubular_imgs[
            np.min(target_mito_coord[0]) - 2 : np.max(target_mito_coord[0]) + 2,
            np.min(target_mito_coord[1]) - 2 : np.max(target_mito_coord[1]) + 2,
            np.min(target_mito_coord[2]) - 2 : np.max(target_mito_coord[2]) + 2
        ],
        0
    )

    merged_imgs = np.concatenate(
        [merged_imgs_mito, merged_imgs_lamellar, merged_imgs_tubular],
        axis=3
    )
    
    out_dir =  f"Z:/paper_data/2021_cristae/correspose_PCA_without_angle/group2/shOPA1_cropped00{CROP_NO}_mito{str(num).zfill(3)}/imgs"
    os.makedirs(out_dir)

    for i in range(merged_imgs.shape[0]):
        cv2.imwrite(
            f"{out_dir}/{str(i).zfill(4)}.png",
            merged_imgs[i]
        )
    
    

del mito_imgs, lamellar_imgs, tubular_imgs, labeled_mito_imgs

In [ ]:
CROP_NO = 5


mito_imgs, _ =  load_Y_gray(f"Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_00{CROP_NO}/devided_mito/", normalize=False)
lamellar_imgs, _ = load_Y_gray(f"Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_00{CROP_NO}/merged_cristae_final/lamellar_with_postprocessing/", normalize=False)
tubular_imgs, _ = load_Y_gray(f"Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_00{CROP_NO}/merged_cristae_final/tubular_with_postprocessing/", normalize=False)

labeled_mito_imgs = cc3d.connected_components(mito_imgs[:,:,:,0].astype(int),  connectivity=6)


lst = list()
for node in group1:
    if node["Gene"] == "shOPA1" and node["cropped_No"] == CROP_NO:
        lst.append(node["mito_No"])
        
for num in lst:
    target_mito_coord = np.nonzero(labeled_mito_imgs==num)
    merged_imgs_mito = np.where(
        labeled_mito_imgs[
            np.min(target_mito_coord[0]) - 2 : np.max(target_mito_coord[0]) + 2,
            np.min(target_mito_coord[1]) - 2 : np.max(target_mito_coord[1]) + 2,
            np.min(target_mito_coord[2]) - 2 : np.max(target_mito_coord[2]) + 2
        ] == num,
        1,
        0
    )[..., np.newaxis]

    merged_imgs_lamellar = np.where(
        merged_imgs_mito == 1,
        lamellar_imgs[
            np.min(target_mito_coord[0]) - 2 : np.max(target_mito_coord[0]) + 2,
            np.min(target_mito_coord[1]) - 2 : np.max(target_mito_coord[1]) + 2,
            np.min(target_mito_coord[2]) - 2 : np.max(target_mito_coord[2]) + 2
        ],
        0
    )

    merged_imgs_tubular = np.where(
        merged_imgs_mito == 1,
        tubular_imgs[
            np.min(target_mito_coord[0]) - 2 : np.max(target_mito_coord[0]) + 2,
            np.min(target_mito_coord[1]) - 2 : np.max(target_mito_coord[1]) + 2,
            np.min(target_mito_coord[2]) - 2 : np.max(target_mito_coord[2]) + 2
        ],
        0
    )

    merged_imgs = np.concatenate(
        [merged_imgs_mito, merged_imgs_lamellar, merged_imgs_tubular],
        axis=3
    )
    
    out_dir =  f"Z:/paper_data/2021_cristae/correspose_PCA_without_angle/group2/shOPA1_cropped00{CROP_NO}_mito{str(num).zfill(3)}/imgs"
    os.makedirs(out_dir)

    for i in range(merged_imgs.shape[0]):
        cv2.imwrite(
            f"{out_dir}/{str(i).zfill(4)}.png",
            merged_imgs[i]
        )
    
    

del mito_imgs, lamellar_imgs, tubular_imgs, labeled_mito_imgs